<a href="https://colab.research.google.com/github/lrhgit/fetalcry/blob/main/damped_harmonic_oscillator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⚙️ Physical Motivation and Model

The model is based on **Newton’s second law**, where the sum of forces acting on a mass equals its acceleration:

$$
m \ddot{x} = -k x - c \dot{x}
$$

Here  
- $m$: mass (inertia)  
- $k$: stiffness (elastic restoring force)  
- $c$: damping coefficient (viscous or resistive losses)  
- $x(t)$: displacement as a function of time  

Rearranging gives the standard **second-order differential equation**:

$$
\ddot{x} + 2\zeta \omega_0 \dot{x} + \omega_0^2 x = 0
$$

with

$$
\omega_0 = \sqrt{\frac{k}{m}}, \qquad
\zeta = \frac{c}{2\sqrt{km}}
$$

where $\omega_0$ is the **natural (undamped) angular frequency**,  
and $\zeta$ (zeta) is the **dimensionless damping ratio**.

## 🌊 Oscillatory Regimes

The dynamic behavior of the system depends on the value of the **damping ratio**, $ \zeta $.
It determines whether the response is oscillatory or purely exponential.

| Regime | Condition | Characteristic Response | Physical meaning |
|:--|:--|:--|:--|
| **Underdamped** | $ \zeta < 1 $ | Oscillatory motion with exponentially decaying amplitude | Energy loss per cycle is small; the system vibrates freely before coming to rest |
| **Critically damped** | $ \zeta = 1 $ | Fastest return to equilibrium without oscillation | System returns to rest as quickly as possible, no overshoot |
| **Overdamped** | $ \zeta > 1 $ | Non-oscillatory, slow exponential decay | Damping dominates; motion is sluggish and aperiodic |

For the **underdamped** case, the displacement can be written as:

$$
x(t) = A_0 e^{-\zeta \omega_0 t} \cos(\omega_d t + \phi)
$$

where

$$
\omega_d = \omega_0 \sqrt{1 - \zeta^2}
$$

is the **damped angular frequency**, slightly lower than the natural frequency $ \omega_0 $.

The exponential term $ e^{-\zeta \omega_0 t} $ defines the **envelope** of the oscillation,
illustrating how energy dissipates over time.  
This rate of decay is directly linked to viscous or structural losses in real systems.

## 💡 Interpretation

Although this is a simple **one-degree-of-freedom (1-DOF)** oscillator,
it captures essential physics of more complex **Fluid–Structure Interaction (FSI)** systems.

- The **mass** $m$ represents inertia, such as tissue or structural mass resisting acceleration.  
- The **stiffness** $k$ represents the restoring elasticity that tends to bring the structure back to its equilibrium shape.  
- The **damping** $c$ captures energy losses — either due to internal tissue viscosity, surrounding fluid drag, or mechanical resistance.  

In the context of **vocal fold dynamics** or **fetal airway modeling**,  
these parameters collectively determine whether oscillations can be **self-sustained**, **damped out**,  
or **critically balanced** near the onset of phonation-like motion.

The damping ratio $ \zeta $ thus provides a compact, dimensionless measure of the balance between  
energy storage (via $k$) and energy dissipation (via $c$).  
Small $ \zeta $ values lead to sustained vibration — a prerequisite for sound generation —  
while high $ \zeta $ suppresses oscillations entirely.

This makes the model a useful conceptual foundation for understanding  
**how viscous and elastic effects influence vibration frequency, amplitude, and decay**  
in FSI problems such as vocal fold or membrane motion.

In [ ]:
#@title 🔧 Interactive oscillator model (click to expand)
#@markdown Run this cell to activate sliders and plots.

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, Checkbox, HBox, VBox, Label, interactive_output


# ---------- Beregningsfunksjon ----------
def mode_response(omega_n, zeta, A_n, t):
    """Returnerer y_n(t) og amplitudekappe (kun for underdempet tilfelle)."""
    if zeta < 1.0:  # underdempet
        omega_d = omega_n * np.sqrt(1.0 - zeta**2)
        A = A_n
        B = (zeta * omega_n * A_n) / omega_d
        y = np.exp(-zeta * omega_n * t) * (A*np.cos(omega_d*t) + B*np.sin(omega_d*t))
        env = A_n * np.exp(-zeta * omega_n * t)  # positiv kappedemping
    elif np.isclose(zeta, 1.0, atol=1e-9):  # kritisk dempet
        A = A_n
        B = omega_n * A_n
        y = (A + B*t) * np.exp(-omega_n * t)
        env = None
    else:  # overdempet
        s = np.sqrt(zeta**2 - 1.0)
        r1 = -omega_n * (zeta + s)
        r2 = -omega_n * (zeta - s)
        A = A_n * r2 / (r2 - r1)
        B = -A_n * r1 / (r2 - r1)
        y = A*np.exp(r1*t) + B*np.exp(r2*t)
        env = None
    return y, env


# ---------- Hovedplot ----------
def mass_spring_harmonics(m, k, c, x0, v0_unused, n_periods, show_decay):
    omega0 = np.sqrt(k/m)
    zeta = c / (2.0*np.sqrt(m*k))
    omega_d = omega0*np.sqrt(1.0 - zeta**2) if zeta < 1.0 else None

    T0 = 2*np.pi / omega0
    t_end = n_periods * T0
    t = np.linspace(0, t_end, 4000)

    colors = plt.cm.viridis(np.linspace(0,1,5))
    plt.figure(figsize=(9,5))

    for n, col in zip(range(0,5), colors):
        omega_n = (n+1)*omega0
        A_n = x0/(n+1)

        if show_decay:
            y_n, env = mode_response(omega_n, zeta, A_n, t)
        else:
            y_n, env = A_n*np.cos(omega_n*t), None

        # signal
        plt.plot(t/T0, y_n, color=col, lw=1.6,
                 label=f"n={n}, ω={omega_n:.2f} rad/s ({omega_n/(2*np.pi):.2f} Hz)")

        # envelope kun for underdempet tilfelle (positiv)
        if (env is not None) and (zeta < 1.0):
            plt.plot(t/T0, env, color=col, ls="--", lw=1)

    # Regimebeskrivelse
    if zeta < 1:
        regime = "Underdempet"
        freq_info = f"ωd={omega_d:.2f} rad/s ({omega_d/(2*np.pi):.2f} Hz)"
        subtitle = ""
    elif np.isclose(zeta, 1.0, atol=1e-9):
        regime = "Kritisk dempet"
        freq_info = "Ingen oscillasjon (aperiodisk retur)"
        subtitle = ""
    else:
        regime = "Overdempet"
        freq_info = "Ingen oscillasjon (to eksponentielle forløp)"
        s = np.sqrt(zeta**2 - 1.0)
        r1 = -omega0*(zeta + s)
        r2 = -omega0*(zeta - s)
        subtitle = f"r₁={r1:.2f}, r₂={r2:.2f} [1/s]"

    plt.title(
        f"ζ={zeta:.3f} | ω₀={omega0:.2f} rad/s ({omega0/(2*np.pi):.2f} Hz) | {freq_info}\n"
        f"{regime} | {'Med demping' if show_decay else 'Uten demping'}",
        fontsize=11
    )
    if subtitle:
        plt.suptitle(subtitle, fontsize=9, y=0.96)

    plt.xlabel("Tid [antall perioder T₀]")
    plt.ylabel("Amplitude (normalisert)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# ---------- Sliders i to kolonner ----------
slider_left = VBox([
    Label(value="🧩 Systemparametre"),
    FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='m [kg]'),
    FloatSlider(value=100.0, min=10.0, max=500.0, step=10.0, description='k [N/m]'),
    FloatSlider(value=0.1, min=0.0, max=10.0, step=0.1, description='c [Ns/m]')
])

slider_right = VBox([
    Label(value="🎛️ Visningskontroll"),
    FloatSlider(value=0.01, min=0.0, max=0.05, step=0.001, description='x₀ [m]'),
    FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='v₀ [m/s]'),
    IntSlider(value=5, min=1, max=20, step=1, description='Antall perioder'),
    Checkbox(value=True, description='Vis demping')
])

ui = HBox([slider_left, slider_right])

# ---------- Koble widgets og funksjon ----------
out = interactive_output(
    mass_spring_harmonics,
    {
        'm': slider_left.children[1],
        'k': slider_left.children[2],
        'c': slider_left.children[3],
        'x0': slider_right.children[1],
        'v0_unused': slider_right.children[2],
        'n_periods': slider_right.children[3],
        'show_decay': slider_right.children[4]
    }
)

display(ui, out)

Output()